In [56]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

_ = load_dotenv()

In [47]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DOC_PATH = "/Users/martijnbeeks/Downloads/TN459590 - EF29 Aankondiging gegunde opdracht - algemene richtlijn, standaardregeling 20240401142856.pdf"
CHROMA_PATH = "/Users/martijnbeeks/Development/Personal/tendergpt/openinkoopdata_appchallenge/chroma"

loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

In [48]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

In [49]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [50]:
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

In [65]:
query = 'Wat zijn de eisen van deze tender?'
docs_chroma = db_chroma.similarity_search_with_score(query, k=5)
context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

# you can use a prompt template
PROMPT_TEMPLATE = """
Antwoord de vraag gebaseerd alleen op basis van de volgende context:
{context}
Beantwoord de vraag op basis van de bovenstaande context: {question}.
Geef een gedetaileerd en concreet antwoord.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

model = ChatOpenAI()
response_text = model.predict(prompt)
print(response_text)

De eisen van deze tender zijn dat de inschrijver moet voldoen aan de CO2-prestatieladder door het indienen van een kopie van het CO2-bewust Certificaat. Daarnaast moet de opdrachtnemer concrete maatregelen kunnen nemen om het beheer, onderhoud en schadeherstel van haltevoorzieningen te optimaliseren, waarbij altijd voldaan moet worden aan het minimale onderhoudsniveau zoals beschreven in het programma van eisen. Verder wordt er gezocht naar een opdrachtnemer die controle en monitoring kan uitvoeren. De waarde van de aanbesteding is geraamd op 3 069 818 EUR en de duur van het contract is van 15 april 2024 tot 14 april 2028, met de mogelijkheid tot verlenging van maximaal 2 keer met 2 jaar. De aanbesteding valt onder de Overeenkomst inzake overheidsopdrachten (GPA) en wordt niet gefinancierd uit EU-fondsen.
